# Bases de Datos Vectoriales y Búsqueda Semántica

## Objetivos de Aprendizaje

En este cuaderno vas a aprender:
- Qué son los embeddings y cómo representan el significado del texto
- Cómo funcionan las bases de datos vectoriales (ChromaDB)
- La diferencia entre búsqueda por palabra clave y búsqueda semántica
- Cómo construir un sistema de búsqueda inteligente en español
- Los fundamentos de la recuperación de información para RAG

## Contexto: El Problema de la Búsqueda Tradicional

Imaginate que tenés una base de datos con miles de reviews de restaurantes. Un usuario busca "lugares con buena comida italiana económica". 

**Búsqueda tradicional** (por palabra clave):
- Solo encuentra documentos que contienen exactamente las palabras "italiana" y "económica"
- Se pierde reviews que dicen "pasta excelente" o "precios accesibles"

**Búsqueda semántica** (por significado):
- Entiende que "pasta" está relacionado con "italiana"
- Relaciona "precios accesibles" con "económica"
- Encuentra documentos relevantes aunque usen palabras diferentes

Esto es posible gracias a los **embeddings** y las **bases de datos vectoriales**.

## Instalación de Dependencias

Vamos a usar:
- **chromadb**: Base de datos vectorial de código abierto
- **sentence-transformers**: Modelos para generar embeddings de texto

In [ ]:
!pip install -qq chromadb
!pip install -q sentence-transformers

In [ ]:
# Verificamos que PyTorch está instalado (necesario para sentence-transformers)
import torch
print(f"PyTorch versión: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## ¿Qué son los Embeddings?

Un **embedding** es una representación numérica (vector) del significado de un texto. 

**Analogía**: Imaginate un mapa de conceptos en un espacio multidimensional:
- Palabras con significados similares están cerca en este espacio
- Palabras con significados diferentes están lejos

Por ejemplo:
- "excelente", "genial", "buenísimo" → vectores cercanos
- "terrible", "pésimo", "horrible" → vectores cercanos entre sí, pero lejos de los anteriores

### Ejemplo Visual

```
"buenísimo"     ●
                 \
"genial"          ●     ← Estos están cerca
                   \
"excelente"         ●


                              ● "terrible"
                             /
                            ● "pésimo"     ← Estos también están cerca
                           /
                          ● "horrible"
```

**En la práctica**: Un embedding es una lista de números (típicamente 384 o 768 dimensiones) que captura el significado del texto.

## ChromaDB: Base de Datos Vectorial

**ChromaDB** es una base de datos especializada en almacenar y buscar vectores de manera eficiente.

### Características principales:
- **Gratuita y open-source**
- **Fácil de usar**: API simple en Python
- **Embeddings automáticos**: Convierte texto a vectores automáticamente
- **Búsqueda por similitud**: Encuentra documentos cercanos en el espacio vectorial
- **Metadatos**: Permite filtrar por campos adicionales

### Operaciones básicas (CRUD):
- **Create**: `collection.add()` - Agregar documentos
- **Read**: `collection.get()` - Obtener documentos por ID
- **Update**: `collection.update()` - Modificar documentos
- **Delete**: `collection.delete()` - Eliminar documentos
- **Query**: `collection.query()` - Buscar por similitud semántica

In [ ]:
import chromadb

# Creamos un cliente de ChromaDB en memoria
# Nota: Los datos se pierden al cerrar el notebook
# Para persistencia, usar: chromadb.PersistentClient(path="./chroma_db")
client = chromadb.Client()

print("ChromaDB inicializado correctamente")

In [ ]:
# Verificamos que no hay colecciones todavía
client.list_collections()

## Ejemplo Práctico: Base de Reviews de Restaurantes

Vamos a crear una base de datos con reviews de restaurantes porteños. Esto nos va a permitir:
- Buscar restaurantes por tipo de comida sin usar palabras exactas
- Encontrar lugares similares aunque se describan diferente
- Recomendar en base a preferencias expresadas en lenguaje natural

### Pasos:
1. Crear una colección
2. Agregar reviews iniciales
3. Probar búsquedas semánticas
4. Comparar con búsqueda tradicional
5. Mejorar con embeddings multilenguaje

In [ ]:
# Creamos nuestra primera colección
# Una colección es como una tabla en SQL, agrupa documentos relacionados
collection = client.create_collection(
    name="reviews_restaurantes"
)

print("Colección 'reviews_restaurantes' creada")

In [ ]:
# Verificamos que la colección existe
client.list_collections()

### Dataset: Reviews de Restaurantes Porteños

Vamos a usar reviews realistas de diferentes tipos de lugares en Buenos Aires.

In [ ]:
# Reviews iniciales - variedad de restaurantes en CABA
reviews_iniciales = [
    "Fui a cenar pasta y la verdad que espectacular. Los ñoquis con salsa fileto increíbles. Precio razonable, como 8 lucas por persona. Ambiente tranquilo, perfecto para ir en pareja.",
    
    "La mejor pizza que comí en mi vida, no jodo. Masa finita, crocante, mucha muzzarella. Eso sí, siempre está lleno y hay que esperar. Vale la pena. Estilo porteño posta.",
    
    "Restaurante gourmet en Palermo, cocina de autor. Los platos son chicos pero re elaborados. Caro pero para una ocasión especial está bueno. Tiene buen vino también.",
    
    "Parrilla clásica de barrio. El bife de chorizo una locura, tierno y jugoso. Las papas fritas caseras. Atención familiar, te hacen sentir como en tu casa. Precios normales.",
    
    "Sushi delivery que pedimos seguido. Fresco, bien armado, llega rápido. No es el mejor que probé pero para el precio está más que bien. El combo para dos es suficiente."
]

# Metadatos: información adicional sobre cada review
metadatos_iniciales = [
    {"barrio": "San Telmo", "tipo": "italiana", "precio": "medio"},
    {"barrio": "Palermo", "tipo": "pizzeria", "precio": "medio"},
    {"barrio": "Palermo", "tipo": "gourmet", "precio": "alto"},
    {"barrio": "Villa Urquiza", "tipo": "parrilla", "precio": "medio"},
    {"barrio": "delivery", "tipo": "sushi", "precio": "medio"}
]

# IDs únicos para cada documento
ids_iniciales = ["review1", "review2", "review3", "review4", "review5"]

### Agregando Documentos a la Colección

Cuando agregamos documentos, ChromaDB automáticamente:
1. Genera embeddings de cada texto usando un modelo por defecto
2. Almacena los vectores en una estructura optimizada para búsqueda
3. Guarda los metadatos asociados

**Nota**: El modelo por defecto (`all-MiniLM-L6-v2`) funciona mejor con inglés. Más adelante vamos a usar un modelo multilenguaje optimizado para español.

In [ ]:
# Agregamos las reviews a la colección
collection.add(
    documents=reviews_iniciales,
    metadatas=metadatos_iniciales,
    ids=ids_iniciales
)

print(f"Se agregaron {len(reviews_iniciales)} reviews a la base de datos")

In [ ]:
# Verificamos cuántos documentos tenemos
collection.count()

In [ ]:
# Obtenemos todos los documentos para verificar
todos = collection.get()
print("Documentos en la colección:")
for i, doc in enumerate(todos['documents'], 1):
    print(f"\n{i}. {doc[:80]}...")

In [ ]:
# También podemos obtener un documento específico por su ID
review_especifica = collection.get(ids=["review2"])
print("Review de pizza:")
print(review_especifica['documents'][0])

## Búsqueda Semántica en Acción

Ahora viene lo interesante: buscar documentos por significado, no por palabras exactas.

### Parámetros de búsqueda:
- **query_texts**: La consulta en lenguaje natural
- **n_results**: Cuántos resultados queremos (los más similares)
- **where**: Filtros opcionales por metadatos

### Cómo funciona internamente:
1. ChromaDB convierte tu consulta en un vector
2. Calcula la distancia (similitud) entre tu consulta y todos los documentos
3. Devuelve los N documentos más cercanos (similares)

In [ ]:
# Búsqueda 1: Quiero comer algo italiano
consulta = "Busco un lugar para comer buena comida italiana, tipo ravioles o ñoquis"

resultados = collection.query(
    query_texts=[consulta],
    n_results=3  # Los 3 más similares
)

print(f"CONSULTA: {consulta}")
print("\nREVIEWS MÁS SIMILARES:")
print("=" * 80)
for i, (doc, metadata) in enumerate(zip(resultados['documents'][0], resultados['metadatas'][0]), 1):
    print(f"\n{i}. {doc}")
    print(f"   Barrio: {metadata['barrio']}, Tipo: {metadata['tipo']}, Precio: {metadata['precio']}")

### Análisis del Resultado

Fijate que **no usamos la palabra "italiana"** en la base de datos original, pero el sistema pudo:
1. Entender que "ñoquis", "ravioles" están relacionados con comida italiana
2. Identificar la review que habla de "pasta" y "ñoquis"
3. Devolver el resultado más relevante

Esto es búsqueda semántica: encuentra por **significado**, no por palabra exacta.

In [ ]:
# Búsqueda 2: Lugar económico para carne
consulta2 = "Dónde puedo ir a comer buen asado sin gastar mucha plata?"

resultados2 = collection.query(
    query_texts=[consulta2],
    n_results=3
)

print(f"CONSULTA: {consulta2}")
print("\nREVIEWS MÁS SIMILARES:")
print("=" * 80)
for i, (doc, metadata) in enumerate(zip(resultados2['documents'][0], resultados2['metadatas'][0]), 1):
    print(f"\n{i}. {doc}")
    print(f"   Barrio: {metadata['barrio']}, Tipo: {metadata['tipo']}, Precio: {metadata['precio']}")

### Comparación: Búsqueda Tradicional vs Semántica

Veamos qué pasa si buscamos con el método tradicional (palabra clave).

In [ ]:
# Búsqueda tradicional: buscar la palabra "asado" exacta
print("BÚSQUEDA TRADICIONAL (palabra exacta 'asado'):")
busqueda_tradicional = collection.get(
    where_document={"$contains": "asado"}
)

if len(busqueda_tradicional['documents']) > 0:
    for doc in busqueda_tradicional['documents']:
        print(f"- {doc}")
else:
    print("No se encontraron resultados con la palabra 'asado'")

print("\n" + "="*80)
print("\nBÚSQUEDA SEMÁNTICA (por significado):")
print("Encontró la parrilla aunque no use la palabra 'asado' exacta")

## Agregando Más Documentos Dinámicamente

En una aplicación real, constantemente llegan nuevas reviews. Veamos cómo agregar documentos de forma dinámica.

In [ ]:
# Nuevas reviews que llegan
nuevas_reviews = [
    "Bar de tragos en Palermo re copado. Buenos cócteles, música en vivo los fines de semana. Se llena bastante después de las 11. Tiene terraza.",
    
    "Cafetería de especialidad, café re rico, tienen opciones veganas. Ambiente tranquilo para trabajar con la compu. WiFi gratis y enchufes.",
    
    "Bodegón español tradicional. Las croquetas y la tortilla de papa son de otro planeta. Vino de la casa riquísimo. Porteño viejo, 100 años de historia.",
    
    "Hamburguesería gourmet. Las papas con cheddar y bacon son adictivas. Burgers de 200gr, jugosas. Podes armar la tuya. Delivery hasta las 2am.",
    
    "Cantina familiar muy casera. La comida es simple pero rica, como la que hace tu nona. Milanesas gigantes. Super económico, 6 lucas comes re bien."
]

nuevos_metadatos = [
    {"barrio": "Palermo", "tipo": "bar", "precio": "medio-alto"},
    {"barrio": "Colegiales", "tipo": "cafeteria", "precio": "medio"},
    {"barrio": "Montserrat", "tipo": "española", "precio": "medio"},
    {"barrio": "Belgrano", "tipo": "hamburguesas", "precio": "medio"},
    {"barrio": "Boedo", "tipo": "casera", "precio": "bajo"}
]

In [ ]:
# Función helper para agregar reviews de forma organizada
def agregar_reviews(collection, reviews, metadatos, prefijo_id="review"):
    """
    Agrega nuevas reviews a una colección existente.
    
    Parámetros:
    -----------
    collection : chromadb.Collection
        La colección donde agregar los documentos
    reviews : list
        Lista de textos de reviews
    metadatos : list
        Lista de diccionarios con metadatos
    prefijo_id : str
        Prefijo para generar IDs únicos
    """
    # Obtenemos cuántos documentos ya hay para generar IDs únicos
    count_actual = collection.count()
    
    # Generamos IDs únicos
    nuevos_ids = [f"{prefijo_id}{count_actual + i + 1}" for i in range(len(reviews))]
    
    # Agregamos a la colección
    collection.add(
        documents=reviews,
        metadatas=metadatos,
        ids=nuevos_ids
    )
    
    print(f"Se agregaron {len(reviews)} nuevas reviews")
    print(f"Total de documentos en la colección: {collection.count()}")
    return nuevos_ids

In [ ]:
# Agregamos las nuevas reviews
ids_nuevos = agregar_reviews(collection, nuevas_reviews, nuevos_metadatos)

In [ ]:
# Probemos una nueva búsqueda con el dataset ampliado
consulta3 = "Lugar tranquilo para tomar café y laburar un rato"

resultados3 = collection.query(
    query_texts=[consulta3],
    n_results=3
)

print(f"CONSULTA: {consulta3}")
print("\nREVIEWS MÁS SIMILARES:")
print("=" * 80)
for i, (doc, metadata) in enumerate(zip(resultados3['documents'][0], resultados3['metadatas'][0]), 1):
    print(f"\n{i}. {doc}")
    print(f"   Barrio: {metadata['barrio']}, Tipo: {metadata['tipo']}, Precio: {metadata['precio']}")

## Mejorando con Embeddings Multilenguaje

El modelo por defecto de ChromaDB (`all-MiniLM-L6-v2`) fue entrenado principalmente con texto en inglés. Para español, especialmente con modismos argentinos, necesitamos un modelo mejor.

### Modelo Recomendado: multilingual-e5-large

**Características**:
- Entrenado en 94 idiomas, incluido español
- Entiende variantes regionales y coloquialismos
- Mejor para textos con vocabulario local ("posta", "re", "trucho", etc.)
- 768 dimensiones (vs 384 del modelo por defecto)

**Trade-off**: Más lento y usa más memoria, pero mucho más preciso para español.

In [ ]:
from chromadb.utils import embedding_functions

# Configuramos el modelo multilenguaje
# Esto descarga el modelo la primera vez (puede tardar unos minutos)
modelo_multilenguaje = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="intfloat/multilingual-e5-large"
)

print("Modelo multilenguaje cargado")
print("Este modelo entiende español argentino mucho mejor")

### Creando una Nueva Colección con Mejor Modelo

Vamos a crear una nueva colección usando el modelo multilenguaje y comparar los resultados.

**Parámetros importantes**:
- `embedding_function`: El modelo que convierte texto a vectores
- `metadata={"hnsw:space": "cosine"}`: Usa similitud coseno para comparar vectores

In [ ]:
# Creamos una nueva colección con el modelo mejorado
collection_mejorada = client.get_or_create_collection(
    name="reviews_restaurantes_multilenguaje",
    embedding_function=modelo_multilenguaje,
    metadata={"hnsw:space": "cosine"}  # Método de comparación de vectores
)

print("Colección con embeddings multilenguaje creada")

In [ ]:
# Verificamos las colecciones disponibles
print("Colecciones en la base de datos:")
for col in client.list_collections():
    print(f"  - {col.name}")

In [ ]:
# Agregamos TODAS las reviews (iniciales + nuevas) a la colección mejorada
todas_las_reviews = reviews_iniciales + nuevas_reviews
todos_los_metadatos = metadatos_iniciales + nuevos_metadatos
todos_los_ids = ids_iniciales + ids_nuevos

collection_mejorada.add(
    documents=todas_las_reviews,
    metadatas=todos_los_metadatos,
    ids=todos_los_ids
)

print(f"Se agregaron {len(todas_las_reviews)} reviews a la colección mejorada")

### Comparación: Modelo Base vs Modelo Multilenguaje

Vamos a hacer la misma búsqueda con ambos modelos y comparar los resultados.

In [ ]:
# Consulta con modismos argentinos
consulta_argentina = "Busco un lugar re copado para morfar algo barato y llenadero"

print("="*80)
print(f"CONSULTA: {consulta_argentina}")
print("="*80)

# Búsqueda con modelo base
print("\n1. MODELO BASE (all-MiniLM-L6-v2):")
print("-"*80)
resultado_base = collection.query(
    query_texts=[consulta_argentina],
    n_results=3
)
for i, doc in enumerate(resultado_base['documents'][0], 1):
    print(f"\n{i}. {doc[:100]}...")

# Búsqueda con modelo multilenguaje
print("\n" + "="*80)
print("\n2. MODELO MULTILENGUAJE (multilingual-e5-large):")
print("-"*80)
resultado_mejorado = collection_mejorada.query(
    query_texts=[consulta_argentina],
    n_results=3
)
for i, (doc, metadata) in enumerate(zip(resultado_mejorado['documents'][0], resultado_mejorado['metadatas'][0]), 1):
    print(f"\n{i}. {doc[:100]}...")
    print(f"   Tipo: {metadata['tipo']}, Precio: {metadata['precio']}")

### Análisis de la Comparación

El modelo multilenguaje entiende mejor:
- **"re copado"** → lugares con buen ambiente
- **"morfar"** → comer
- **"barato y llenadero"** → buena relación precio/cantidad

Esto resulta en recomendaciones más relevantes para usuarios argentinos.

## Experimentación: Zona de Pruebas

Probá tus propias consultas y observá cómo funciona la búsqueda semántica.

In [ ]:
# TU TURNO: Modifica esta consulta
MI_CONSULTA = "Quiero ir a comer algo rico pero sin gastar mucho"

# Cantidad de resultados que querés ver
CANTIDAD_RESULTADOS = 3

# Ejecutamos la búsqueda
mis_resultados = collection_mejorada.query(
    query_texts=[MI_CONSULTA],
    n_results=CANTIDAD_RESULTADOS
)

print(f"CONSULTA: {MI_CONSULTA}")
print("\nRECOMENDACIONES:")
print("="*80)
for i, (doc, metadata) in enumerate(zip(mis_resultados['documents'][0], mis_resultados['metadatas'][0]), 1):
    print(f"\n{i}. {doc}")
    print(f"   Barrio: {metadata['barrio']}, Tipo: {metadata['tipo']}, Precio: {metadata['precio']}")

### Ejercicios Sugeridos

Probá estas búsquedas y observá los resultados:

1. "Un lugar romántico para llevar a mi pareja"
2. "Dónde puedo comer algo rápido al mediodía"
3. "Restaurante con opciones vegetarianas"
4. "Lugar con buena onda para ir con amigos"
5. "Comida casera como la que hace mi abuela"

## Conexión con RAG (Próximo Paso)

Lo que hicimos hoy es la **primera mitad de RAG**: la parte de **recuperación** (Retrieval).

### RAG = Retrieval (Recuperar) + Augmented Generation (Generar con Contexto)

**Lo que ya sabemos hacer** (este cuaderno):
1. Almacenar documentos con sus embeddings
2. Buscar documentos similares a una consulta
3. Obtener los más relevantes

**Lo que vamos a aprender** (próximo cuaderno):
4. Tomar los documentos encontrados
5. Pasárselos como contexto a un LLM (GPT/Gemini)
6. Generar una respuesta personalizada basada en esos documentos

### Ejemplo de RAG Completo

**Usuario pregunta**: "Recomendame un lugar para comer pasta buena y económica"

**Paso 1 - RETRIEVAL** (lo que ya sabemos):
- Buscar en nuestra base vectorial
- Encontrar: Review de la trattoria con ñoquis, review de la cantina familiar

**Paso 2 - GENERATION** (próxima clase):
- Prompt a Gemini: "Basándote en estas reviews: [contexto], recomienda un lugar para pasta económica"
- Gemini genera: "Te recomiendo la cantina familiar en Boedo. Según las reviews, la comida es casera y las porciones son generosas..."

### ¿Por qué es Poderoso RAG?

1. **Información específica**: El LLM usa TUS datos, no solo su conocimiento general
2. **Actualizable**: Agregás nuevas reviews y el sistema las usa inmediatamente
3. **Verificable**: Podés mostrar qué documentos se usaron para generar la respuesta
4. **Privado**: Tus datos quedan en tu base, no se envían para entrenar modelos

In [ ]:
# Simulación de cómo funcionará RAG (próxima clase)
def simular_rag(consulta_usuario):
    """
    Simulación simple de un sistema RAG completo.
    En la próxima clase implementaremos la parte de generación con Gemini.
    """
    print("=" * 80)
    print("SIMULACIÓN DE SISTEMA RAG")
    print("=" * 80)
    print(f"\nConsulta del usuario: {consulta_usuario}")
    
    # PASO 1: RETRIEVAL (ya lo sabemos hacer)
    print("\n[PASO 1] RETRIEVAL - Buscando documentos relevantes...")
    resultados = collection_mejorada.query(
        query_texts=[consulta_usuario],
        n_results=2
    )
    
    docs_relevantes = resultados['documents'][0]
    print(f"Se encontraron {len(docs_relevantes)} documentos relevantes:")
    for i, doc in enumerate(docs_relevantes, 1):
        print(f"  {i}. {doc[:80]}...")
    
    # PASO 2: GENERATION (próxima clase con Gemini)
    print("\n[PASO 2] GENERATION - Generando respuesta personalizada...")
    print("(Próxima clase: usaremos Gemini/GPT con este contexto)")
    print("\nRespuesta simulada:")
    print("Basándome en las reviews encontradas, te recomiendo...")
    print("[Aquí Gemini generaría una respuesta natural usando los documentos]")
    
    return docs_relevantes

# Probemos la simulación
simular_rag("Quiero un lugar barato para comer mucho")

## Resumen y Conceptos Clave

### Lo que aprendimos:

1. **Embeddings**: Representaciones numéricas del significado del texto
   - Palabras similares tienen vectores similares
   - Permiten comparar textos por significado, no solo por palabras

2. **ChromaDB**: Base de datos vectorial
   - Almacena embeddings de forma eficiente
   - Operaciones CRUD estándar
   - Búsqueda por similitud semántica

3. **Búsqueda Semántica vs Tradicional**:
   - Tradicional: Solo encuentra palabras exactas
   - Semántica: Entiende sinónimos, contexto, significado

4. **Modelos de Embeddings**:
   - `all-MiniLM-L6-v2`: Rápido, bueno para inglés
   - `multilingual-e5-large`: Mejor para español, modismos, regionalismos

5. **Parámetros Configurables**:
   - `n_results`: Cantidad de documentos a devolver
   - `embedding_function`: Modelo que genera los vectores
   - `metadata`: Información adicional para filtrar

### Próximos pasos:

En el siguiente cuaderno vamos a:
- Integrar ChromaDB con Gemini/GPT
- Construir un sistema RAG completo
- Generar respuestas contextualizadas basadas en nuestros documentos
- Crear un asistente que responde usando información específica

## Glosario

**Embedding**: Vector numérico que representa el significado de un texto. Textos con significados similares tienen embeddings similares.

**Vector**: Lista de números (típicamente 384 o 768 valores) que representa un punto en un espacio multidimensional.

**Base de Datos Vectorial**: Sistema especializado en almacenar y buscar vectores de forma eficiente (ej: ChromaDB, Pinecone, Weaviate).

**Similitud Coseno**: Medida de qué tan similar es la dirección de dos vectores. Valor entre -1 (opuestos) y 1 (idénticos).

**Búsqueda Semántica**: Búsqueda por significado en lugar de coincidencia exacta de palabras.

**Colección**: Grupo de documentos relacionados en ChromaDB, similar a una tabla en SQL.

**Metadatos**: Información adicional sobre un documento (ej: fecha, autor, categoría) que no es parte del texto principal.

**RAG (Retrieval Augmented Generation)**: Técnica que combina búsqueda de información relevante con generación de texto usando LLMs.

**Query**: Consulta o pregunta que se hace a la base de datos.

**HNSW (Hierarchical Navigable Small World)**: Algoritmo eficiente para búsqueda de vecinos más cercanos en espacios de alta dimensión.

## Preguntas Frecuentes

**P: ¿Por qué usar una base de datos vectorial en lugar de una base de datos normal?**

R: Las bases vectoriales están optimizadas para buscar por similitud semántica. Una SQL busca coincidencias exactas; una vectorial encuentra "lo más parecido" incluso si las palabras son diferentes.

---

**P: ¿Cuánto espacio ocupan los embeddings?**

R: Depende del modelo. multilingual-e5-large usa 768 dimensiones × 4 bytes = ~3KB por documento. Para 1 millón de documentos serían ~3GB.

---

**P: ¿Puedo usar ChromaDB en producción?**

R: Sí, pero para aplicaciones grandes considerá alternativas como Pinecone o Weaviate que están más optimizadas para escala. ChromaDB es excelente para prototipos y aplicaciones medianas.

---

**P: ¿Qué pasa si mi texto tiene más de 512 tokens?**

R: Los modelos de embeddings tienen un límite (típicamente 512 tokens). Si tu documento es más largo, tenés que dividirlo en chunks. Lo vemos en el próximo cuaderno.

---

**P: ¿Los embeddings se actualizan cuando cambio un documento?**

R: No, los embeddings son estáticos. Si modificás un documento, tenés que regenerar su embedding usando `collection.update()`.

## Referencias y Recursos

**Documentación**:
- ChromaDB Docs: https://docs.trychroma.com/
- Sentence Transformers: https://www.sbert.net/
- Modelo multilingual-e5: https://huggingface.co/intfloat/multilingual-e5-large

**Papers Relevantes**:
- "Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks"
- "Text Embeddings by Weakly-Supervised Contrastive Pre-training" (E5)
- "Efficient and Robust Approximate Nearest Neighbor Search Using HNSW"

**Recursos Adicionales**:
- Lista de modelos de embeddings: https://huggingface.co/spaces/mteb/leaderboard
- Tutorial interactivo de embeddings: https://projector.tensorflow.org/